In [14]:
#Importation des bibliothèques

from dotenv import load_dotenv
import os
import sys
import requests
import json
import logging
import time
import pandas as pd
from pandas import json_normalize
import geopandas as gpd
from io import StringIO

In [15]:
load_dotenv()  # Charge les variables d'environnement du fichier .env

APPLICATION_ID= os.getenv("APPLICATION_ID")


In [16]:
import json
import requests
import time


logging.captureWarnings(True)
# Example of a Python implementation for a continuous authentication client.
# It's necessary to :
# - update APPLICATION_ID
# - update request_url at the end of the script

# unique application id : you can find this in the curl's command to generate jwt token 
APPLICATION_ID = 'Q2Y4QzFxODZ6Ukd4VWhjNzY0aWpOc1NmM1dFYTp3OGdxNXhhMDY5dDl0OXJudFdPYjRveGpzbzBh'

# url to obtain acces token
TOKEN_URL = "https://portail-api.meteofrance.fr/token"

class Client(object):

    def __init__(self):
        self.session = requests.Session()

    def request(self, method, url, **kwargs):
        # First request will always need to obtain a token first
        if 'Authorization' not in self.session.headers:
            self.obtain_token()

        # Optimistically attempt to dispatch reqest
        response = self.session.request(method, url, **kwargs)
        if self.token_has_expired(response):
            # We got an 'Access token expired' response => refresh token
            self.obtain_token()
            # Re-dispatch the request that previously failed
            response = self.session.request(method, url, **kwargs)

        return response

    def token_has_expired(self, response):
        status = response.status_code
        content_type = response.headers['Content-Type']
        repJson = response.text
        if status == 401 and 'application/json' in content_type:
            repJson = response.text
            if 'Invalid JWT token' in repJson['description']:
                return True
        return False

    def obtain_token(self):
        # Obtain new token
        data = {'grant_type': 'client_credentials'}
        headers = {'Authorization': 'Basic ' + APPLICATION_ID}
        access_token_response = requests.post(TOKEN_URL, data=data, verify=False, allow_redirects=False, headers=headers)
        token = access_token_response.json()['access_token']
        # Update session with fresh token
        self.session.headers.update({'Authorization': 'Bearer %s' % token})

client = Client()
 # Issue a series of API requests an example. For use this test, you must first subscribe to the arome api with your application
client.session.headers.update({'Accept': 'application/json'})


In [17]:
#on récupère la liste des stations
station=client.request('GET', 'https://public-api.meteofrance.fr/public/DPObs/v1/liste-stations', verify=False)



data = StringIO(station.text)
df = pd.read_csv(data,sep=';')


In [18]:
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs=4326
)
gdf

,Id_station,Id_omm,Nom_usuel,Latitude,Longitude,Altitude,Date_ouverture,Pack,geometry
0,1014002,NaN,ARBENT,46.278167,5.669000,534,2003-10-01,RADOME,POINT (5.669 46.27817)
1,1027003,NaN,BALAN_AERO,45.833000,5.106667,196,2014-05-26,ETENDU,POINT (5.10667 45.833)
2,1034004,NaN,BELLEY,45.769333,5.688000,330,2001-09-13,RADOME,POINT (5.688 45.76933)
3,1064001,NaN,VERIZIEU,45.777167,5.487167,281,2015-01-01,ETENDU,POINT (5.48717 45.77717)
4,1071001,NaN,CESSY,46.310333,6.080333,507,2002-05-01,RADOME,POINT (6.08033 46.31033)
...,...,...,...,...,...,...,...,...,...
2116,98832004,91588.0,MTGNE SOURCES,-22.143833,166.593167,773,1989-08-01,ETENDU,POINT (166.59317 -22.14383)
2117,98832005,NaN,OUINNE,-21.984000,166.680500,54,1974-01-01,ETENDU,POINT (166.6805 -21.984)
2118,98832006,NaN,RIVIERE BLANCHE,-22.132667,166.726333,171,2000-11-01,ETENDU,POINT (166.72633 -22.13267)
2119,98832101,NaN,GORO_ANCIENNE_PEPINIERE,-22.269167,166.967500,298,1995-01-01,ETENDU,POINT (166.9675 -22.26917)


In [19]:
from cartiflette import carti_download
departements = carti_download(
    values="France",
    crs=4326,
    borders="DEPARTEMENT",
    vectorfile_format="geojson",
    filter_by="FRANCE_ENTIERE_DROM_RAPPROCHES",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022,
)



In [20]:
#On récupere la liste des stations climatologiques envoant les mesures à une fréquence de 1 journée
client = Client()
 # Issue a series of API requests an example. For use this test, you must first subscribe to the arome api with your application
client.session.headers.update({'Accept': 'application/json'})
response = client.request('GET', 'https://public-api.meteofrance.fr/public/DPClim/v1/liste-stations/infrahoraire-6m?id-departement=91', verify=False)
df=pd.DataFrame(response.json())
print(df.head())





         id            nom  posteOuvert  typePoste       lon        lat  alt  \
0  91027002           ORLY         True          0  2.397000  48.718000   86   
1  91069001    BOIGNEVILLE        False          2  2.381167  48.325500  115   
2  91103001  BRETIGNY_SAPC         True          2  2.326000  48.600500   78   
3  91184001   COURDIMANCHE         True          1  2.374333  48.421833   71   
4  91200002        DOURDAN         True          1  1.993000  48.526167  116   

   postePublic  
0         True  
1         True  
2         True  
3         True  
4         True  


In [21]:
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lon, df.lat), crs=4326
)
gdf

gdf[["lon", "lat"]].mean().values.tolist()

[2.2518703333333336, 48.56851844444444]

In [22]:
import seaborn as sns
import folium


def interactive_map_dpe(dpe):

    center = dpe[["lon", "lat"]].mean().values.tolist()
    sw = dpe[["lon", "lat"]].min().values.tolist()
    ne = dpe[["lon", "lat"]].max().values.tolist()

    m = folium.Map(location=center, tiles="OpenStreetMap")

    # I can add markers one by one on the map
    for i in range(0, len(dpe)):
        folium.Marker(
            [dpe.iloc[i]["lat"], dpe.iloc[i]["lon"]],
            icon=folium.Icon(
                color="black", icon="cloud"
            ),
        ).add_to(m)

    m.fit_bounds([sw, ne])

    return m


m = interactive_map_dpe(gdf)

In [23]:
m


In [27]:
def gener_idcommande(datedebut,datefin):

   client=Client()
   response1 = client.request('GET', 'https://public-api.meteofrance.fr/public/DPClim/v1/commande-station/quotidienne?id-station=01014002&date-deb-periode={}T14%3A30%3A00Z&date-fin-periode={}T14%3A30%3A00Z'.format(datedebut,datefin), verify=False)
   response1=response1.json()
   #print(response1)
   return int((response1.get('elaboreProduitAvecDemandeResponse')).get('return'))




gener_idcommande("2024-11-01","2024-11-05")


785316809547

In [33]:
client=Client()
datameteo=client.request('GET',"https://public-api.meteofrance.fr/public/DPClim/v1/commande/fichier?id-cmde={}".format(gener_idcommande("2024-11-01","2024-11-05")),verify=False)


print(datameteo)
data = StringIO(datameteo.text)
df = pd.read_csv(data,sep=';')
print(df)


<Response [201]>
     POSTE      DATE   RR  DRR    TN  HTN    TX   HTX   TM   TMNX  ...  BRUME  \
0  1014002  20241101  0,0  NaN   1,2  614  21,8  1238  8,6  11,50  ...    NaN   
1  1014002  20241102  0,2  NaN  -0,3  510  18,1  1225  7,7   8,90  ...    NaN   
2  1014002  20241103  0,0  NaN   3,6  728  19,2  1356  8,4  11,40  ...    NaN   
3  1014002  20241104  0,2  NaN   0,4  657  20,9  1252  8,1  10,65  ...    NaN   
4  1014002  20241105  0,0  NaN   2,1  323  20,8  1216  9,4  11,45  ...    NaN   

   ECLAIR  ETPMON ETPGRILLE  UV  TMERMAX  TMERMIN HNEIGEF NEIGETOTX  \
0     NaN     NaN       1,3 NaN      NaN      NaN     NaN         0   
1     NaN     NaN       0,8 NaN      NaN      NaN     NaN         0   
2     NaN     NaN       1,2 NaN      NaN      NaN     NaN         0   
3     NaN     NaN       1,2 NaN      NaN      NaN     NaN         0   
4     NaN     NaN       1,2 NaN      NaN      NaN     NaN         0   

   NEIGETOT06  
0           0  
1           0  
2           0  
3    